In [ ]:
import os
import pandas as pd
from pathlib import Path
import tempfile
import cv2
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import sys
sys.path.append('./')
from videollama2 import model_init, mm_infer
from videollama2.utils import disable_torch_init

def get_video_duration(video_path):
    """Get the duration of a video in seconds."""
    try:
        clip = VideoFileClip(video_path)
        duration = clip.duration
        clip.close()
        return duration
    except Exception as e:
        raise Exception(f"Error getting video duration: {str(e)}")

def sample_frames_dynamic(video_path, output_path, target_frames=30):
    """Sample frames from video to create a shorter version."""
    try:
        cap = cv2.VideoCapture(video_path)
        fps = 1
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        sampling_interval = max(1, int(total_frames / target_frames))
        print(f"Sampling every {sampling_interval} frames to extract {target_frames} frames.")
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        saved_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % sampling_interval == 0 and saved_count < target_frames:
                out.write(frame)
                saved_count += 1
            frame_count += 1
            if saved_count >= target_frames:
                break
        
        cap.release()
        out.release()
    except Exception as e:
        raise Exception(f"Error sampling and recreating video: {str(e)}")

def process_video_inference(video_path):
    """Run VideoLLaMA2 inference on a single video."""
    try:
        disable_torch_init()

        few_shot_prompt = """
        Example 1:
        Input: A video showing a person putting an item into their bag and walking out without paying.
        Instruction: Is there an indication of suspicious store activity in this video such as shoplifting or not. Reply only with Yes or No
        Output: Yes

        Example 2:
        Input: A video showing a person examining items, putting one back, and proceeding to the checkout.
        Instruction: Is there an indication of suspicious store activity in this video such as shoplifting or not. Reply only with Yes or No
        Output: No

        Example 3:
        Input: A video showing a person concealing an item in their coat while looking around nervously.
        Instruction: Is there an indication of suspicious store activity in this video such as shoplifting or not. Reply only with Yes or No
        Output: Yes
        
        Now analyze this input:
        """
        
        instruct = """
            Is there an indication of suspicious store activity in this video such as shoplifting or not. 
            Reply only with Yes or No
        """

        combined_instruct = f"{few_shot_prompt}\nInstruction: {instruct}"
        
        model_path = 'DAMO-NLP-SG/VideoLLaMA2.1-7B-16F'
        model, processor, tokenizer = model_init(model_path)
        
        output = mm_infer(
            processor['video'](video_path), 
            combined_instruct, 
            model=model, 
            tokenizer=tokenizer, 
            do_sample=False, 
            modal='video'
        )
        
        return output.strip()
    except Exception as e:
        raise Exception(f"Error in video inference: {str(e)}")

def process_video_directory(video_dir, output_csv, max_duration=30):
    """Process all videos in directory and save results to CSV."""
    results = []
    temp_dir = tempfile.mkdtemp()
    video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    # Initialize model and processor once outside the loop
    print("Initializing VideoLLaMA2 model...")
    model_path = 'DAMO-NLP-SG/VideoLLaMA2-7B'
    model, processor, tokenizer = model_init(model_path)
    
    for video_file in tqdm(video_files, desc="Processing videos"):
        try:
            video_path = os.path.join(video_dir, video_file)
            print(f"\nProcessing {video_file}...")
            
            duration = get_video_duration(video_path)
            print(f"Video duration: {duration:.2f} seconds")
            
            # Handle long videos by sampling frames
            if duration > max_duration:
                sampled_frames_path = os.path.join(temp_dir, f"resampled_{video_file}")
                sample_frames_dynamic(video_path, sampled_frames_path)
                process_path = sampled_frames_path
                print("Frame sampling complete")
            else:
                process_path = video_path
            
            # Run inference
            answer = process_video_inference(process_path)
            
            results.append({
                'filename': video_file,
                'duration': duration,
                'trimmed': duration > max_duration,
                'anomaly': answer,
            })
            
            print(f"Analysis complete - Anomaly detected: {answer}")
            
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            results.append({
                'filename': video_file,
                'duration': -1,
                'trimmed': False,
                'anomaly': 'ERROR',
                'error': str(e)
            })

    # Cleanup
    try:
        import shutil
        shutil.rmtree(temp_dir)
        print(f"\nCleaned up temporary directory: {temp_dir}")
    except Exception as e:
        print(f"Error cleaning up temporary directory: {str(e)}")

    # Save results
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\nResults saved to {output_csv}")
    return df

VIDEO_DIR = "/home/ubuntu/content/drive/MyDrive/shoplifting-videos"  
OUTPUT_CSV = "videollama_results.csv"
MAX_DURATION = 30

results_df = process_video_directory(VIDEO_DIR, OUTPUT_CSV, MAX_DURATION)

print("\nAnalysis Summary:")
print(f"Total videos processed: {len(results_df)}")
print(f"Videos trimmed: {len(results_df[results_df['trimmed']])}")
print(f"Anomalies detected: {len(results_df[results_df['anomaly'] == 'Yes'])}")
print(f"Processing errors: {len(results_df[results_df['anomaly'] == 'ERROR'])}")